# 2:1 MENT

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import zuko

import ment_torch as ment

In [ ]:
plt.style.use("style.mplstyle")

## Settings

In [ ]:
dist_name = "galaxy"
ndim = 2
n_meas = 6
n_bins = 80
xmax = 6.0
seed = 0

## Source distribution

In [ ]:
dist = ment.dist.get_dist(dist_name, ndim=ndim, seed=seed, normalize=True)
x_true = dist.sample(1_000_000)
x_true = x_true.float()

In [ ]:
limits = 2 * [(-xmax, xmax)]

fig, axs = plt.subplots(ncols=2, figsize=(6.0, 3.0))
for i, ax in enumerate(axs):
    hist, edges = np.histogramdd(x_true.numpy(), bins=75, range=limits)
    if i:
        hist = np.log10(hist + 1.00e-15)        
    ax.pcolormesh(edges[0], edges[1], hist.T, vmin=(-3.0 if i else 0.0))

## Forward model

In [ ]:
phase_advances = torch.linspace(0.0, np.pi, n_meas + 1)[:-1]

transfer_matrices = []
for phase_advance in phase_advances:
    matrix = ment.utils.rotation_matrix(phase_advance)
    matrix = matrix.float()
    transfer_matrices.append(matrix)

transforms = []
for matrix in transfer_matrices:
    transform = ment.sim.LinearTransform(matrix)
    transforms.append(transform)

bin_edges = torch.linspace(-xmax, xmax, n_bins + 1)

diagnostics = []
for transform in transforms:
    diagnostic = ment.Histogram1D(axis=0, edges=bin_edges, blur=1.0)
    diagnostics.append([diagnostic])

projections = ment.simulate_with_diag_update(x_true, transforms, diagnostics, blur=0.0)

## Reconstruction model

In [ ]:
prior = ment.GaussianPrior(ndim=2, scale=1.0)

In [ ]:
flow = zuko.flows.NSF(features=ndim, transforms=5, hidden_features=[64] * 3)
flow = zuko.flows.Flow(flow.transform.inv, flow.base)  # fast sampling

sampler = ment.samp.FlowSampler(
    ndim=ndim, 
    flow=flow, 
    unnorm_matrix=torch.linalg.cholesky(torch.eye(ndim)), 
    train_kws=dict(
        iters=1000,
        batch_size=512,
    )
)

In [ ]:
model = ment.MENT(
    ndim=ndim,
    transforms=transforms,
    projections=projections,
    prior=prior,
    sampler=sampler,
    nsamp=100_000,
    verbose=1,
    mode="forward",
)

## Training

In [ ]:
def plot_model(model):
    figs = []

    # Plot reconstructed vs. true distribution.
    x_pred = model.sample(1_000_000)
    projections_true = ment.unravel(model.projections)
    projections_pred = ment.unravel(ment.simulate(x_pred, model.transforms, model.diagnostics))

    fig, axs = plt.subplots(ncols=2, figsize=(6.0, 3.0))
    for i, ax in enumerate(axs):
        hist, edges = np.histogramdd(x_pred.numpy(), bins=75, range=limits)
        if i:
            hist = np.log10(hist + 1.00e-15)        
        ax.pcolormesh(edges[0], edges[1], hist.T, vmin=(-3.0 if i else 0.0))
    figs.append(fig)

    # Plot simulated vs. measured projections.
    ncols = min(n_meas, 7)
    nrows = int(np.ceil(n_meas / ncols))

    for log in [False, True]:
        fig, axs = plt.subplots(
            ncols=ncols, 
            nrows=nrows, 
            figsize=(1.90 * ncols, 1.25 * nrows),
            sharex=True,
            sharey=True,
        )
        for index in range(len(projections_true)):
            ax = axs[index]
            proj_true = projections_true[index]
            proj_pred = projections_pred[index]
            scale = proj_true.values.max()
            ax.plot(proj_true.coords, proj_true.values / scale, color="lightgray")
            ax.plot(proj_pred.coords, proj_pred.values / scale, color="black", marker=".", lw=0, ms=1.0)
            ax.set_ylim(ax.get_ylim()[0], 1.25)
            ax.set_xlim(-xmax, xmax)
            if log:
                ax.set_yscale("log")
                ax.set_ylim(1.00e-05, 5.0)
        figs.append(fig)

    fig, ax = plt.subplots(figsize=(3, 2))
    ax.plot(model.sampler.results["loss"])
    figs.append(fig)

    return figs

In [ ]:
for epoch in range(5):
    if epoch > 0:
        model.gauss_seidel_step(learning_rate=0.75)

    for fig in plot_model(model):
        plt.show()